# Postman Security Incident Response Notebook Template
## Table of Contents
1. [ADS Template](#ADS-Template)
2. [Playbook](#Playbook)
3. [Runbook](#Runbook)

# ADS Template
## ADS Template
### Goal
The goal is the intended purpose of the alert.
### Categorization
Mapping to the MITRE ATT&CK Framework.
### Strategy Abstract
High-level walkthrough of how the ADS functions.
### Technical Context
Detailed information and background needed for a responder.
### Blind Spots and Assumptions
Recognized issues, assumptions, and areas where an ADS may not fire.
### False Positives
Known instances of an ADS misfiring and how to minimize them.
### Validation
Steps required to generate a true positive event.
### Priority
Various alerting levels that an ADS may have.
### Response
General response steps if the alert fires.
### Additional Resources
Any other useful references for understanding the ADS.

# Playbook
This section outlines the playbook for responding to security incidents.

# Runbook
This section contains all the boto3 code and Athena searches for automated incident response.

# Postman Security Jupyter Notebook Template
## Versioning and Metadata
Version: 1.0
Last Updated: 2023-09-11
Author: @YourName
Reviewers: [Names of Reviewers]
## Table of Contents
1. [Initial Response](#Initial-Response)
2. [Investigation](#Investigation)
3. [Athena Queries](#Athena-Queries)
4. [Containment](#Containment)
5. [Eradication](#Eradication)
6. [Recovery](#Recovery)
7. [Lessons Learned](#Lessons-Learned)
8. [Appendices](#Appendices)

## Initial Response
### Objective
Quickly assess the situation and determine the scope of the incident.
### Steps
1. Validate the alert in the Security Console.
2. Identify the affected instance and its metadata tags.
3. Move into the investigation phase.

In [ ]:
import boto3
import pandas as pd
from pyathena import connect
from IPython.display import display

athena_bucket = 's3://your-athena-bucket/'
region = 'your-region'

def query_athena(sql):
    cursor = connect(s3_staging_dir=athena_bucket, region_name=region).cursor()
    cursor.execute(sql)
    columns = cursor.description
    data = cursor.fetchall()
    column_names = [column[0] for column in columns]
    rows = [dict(zip(column_names, data_row)) for data_row in data]
    df = pd.DataFrame(rows, columns=column_names)
    display(df)

## Investigation
### Objective
Understand the nature of the access and whether it was malicious.
### Athena Queries
#### Query 1: Identify Relevant Events
```sql
SELECT eventTime, eventName, sourceIPAddress, userAgent
FROM cloudtrail_logs
WHERE eventName = 'YourEventName'
AND eventSource = 'YourEventSource'
AND instanceId = '<Affected_Instance_ID>'
ORDER BY eventTime DESC;
```
#### Query 2: Identify User Activities
```sql
SELECT eventTime, eventName, eventSource, sourceIPAddress, userAgent
FROM cloudtrail_logs
WHERE userIdentity.arn = '<Affected_User_ARN>'
AND eventName IN ('AssumeRole', 'AddUserToGroup', 'AttachUserPolicy')
AND eventTime BETWEEN '<Start_Time>' AND '<End_Time>'
ORDER BY eventTime DESC;
```

In [ ]:
# Query to identify relevant events
sql_query1 = """
SELECT eventTime, eventName, sourceIPAddress, userAgent
FROM cloudtrail_logs
WHERE eventName = 'YourEventName'
AND eventSource = 'YourEventSource'
AND instanceId = '<Affected_Instance_ID>'
ORDER BY eventTime DESC
"""
query_athena(sql_query1)

# Query to identify user activities
sql_query2 = """
SELECT eventTime, eventName, eventSource, sourceIPAddress, userAgent
FROM cloudtrail_logs
WHERE userIdentity.arn = '<Affected_User_ARN>'
AND eventName IN ('AssumeRole', 'AddUserToGroup', 'AttachUserPolicy')
AND eventTime BETWEEN '<Start_Time>' AND '<End_Time>'
ORDER BY eventTime DESC
"""
query_athena(sql_query2)

## Containment
### Objective
Isolate the affected instances to prevent further damage.
### Steps
1. Isolate the affected instance in a security group.
2. Disable any suspicious IAM roles or policies.
3. Take snapshots of affected volumes for forensic analysis.

## Eradication
### Objective
Remove the root cause of the incident.
### Steps
1. Terminate any unauthorized instances.
2. Delete or modify any unauthorized security groups, IAM roles, or policies.
3. Validate that the root cause has been removed.

## Recovery
### Objective
Restore and validate system functionality for business operations to resume.
### Steps
1. Monitor the affected systems for signs of weaknesses that could be re-exploited.
2. Restore affected systems from clean backups.
3. Confirm with business stakeholders that system functionality is restored.

## Lessons Learned
### Objective
Review the incident to identify areas for improvement.
### Steps
1. Conduct a post-incident review with all involved parties.
2. Document any changes to procedures, policies, and tools.
3. Update the ADS and other detection mechanisms based on the findings.

## Appendices
### A. Additional Resources
1. [AWS Athena Documentation](https://docs.aws.amazon.com/athena/latest/ug/what-is.html)
2. [MITRE ATT&CK Framework](https://attack.mitre.org/)
3. [Your Company's Security Policies](#)
### B. Glossary
1. ADS - Alert Detection Strategy
2. IAM - Identity and Access Management
3. SIEM - Security Information and Event Management